In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import sys
import xesmf as xe
import pandas as pd
from dateutil.relativedelta import relativedelta
import dask

In [2]:
ystart=1970
yend=2020
initmon=2
initmonstr=str(initmon).zfill(2)
endmon=7

In [3]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '30GB',
    processes = 1,
    queue = 'casper',
    local_directory='$TMPDIR',
    resource_spec='select=1:ncpus=1:mem=30GB',
    project='P04010022',
    walltime='02:00:00',
    interface='ext')

# scale up
cluster.scale(24)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/u/home/islas/miniconda3/envs/islaenv24/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39687 instead
  f"Port {expected} is already in use.\n"


In [4]:
cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/39687/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.210:41061,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/39687/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Read in the ERA5 data

In [5]:
dat = xr.open_mfdataset("/glade/campaign/cgd/cas/observations/JRA3Q/mon/ua/*.nc")

## Loop through and grab out the relevant levels and longitude

In [6]:
cesm_grid = xr.open_dataset("/glade/campaign/cesm/collections/CESM2-LE/atm/proc/tseries/month_1/PHIS/"
     +"b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h0.PHIS.185001-185912.nc").isel(time=0).load()

In [8]:
alldat=[]
for iyear in np.arange(ystart,yend+1,1):
    print(iyear)
    datuse = dat.sel(time=slice(str(iyear)+'-'+initmonstr+'-01', str(iyear)+'-'+str(endmon).zfill(2)+'-31'))
    datuse = datuse.sel(pre=slice(100,400)).sel(lon=120., method='nearest')
    datuse = datuse.interp(lat=cesm_grid.lat.values)
    if (iyear == ystart):
        datestart = pd.to_datetime("1970-"+str(datuse.isel(time=0).time.dt.month.values).zfill(2)+"-"+str(datuse.isel(time=0).time.dt.day.values).zfill(2),
                                   format='%Y-%m-%d')
        time = [ datestart + relativedelta(months = int(i)) for i in np.arange(0,datuse.time.size,1) ]
    datuse['time'] = time
    alldat.append(datuse)
alldat = xr.concat(alldat, dim='init_year')
alldat['init_year'] = np.arange(ystart,yend+1,1)

1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [9]:
alldat.to_netcdf("/glade/campaign/cgd/cas/islas/python_savs/NCAR_CW3E_SMYLE/DATA_SORT/U_TEJ/U_400_100_120E_JRA3Q_init"+initmonstr+".nc")

In [10]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
